# Notebook for Climate Data 

In [6]:
import xarray as xr
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import common_paths

# Load Climate Modell

data_pr = xr.open_dataset(common_paths.CLIMA_MODEL.joinpath("PR_data_climatemodel.nc"))

In [7]:
# Variablen und Datenpunkte
data_pr.dims
 

FrozenMappingWarningOnValuesAccess({'time': 1140, 'bnds': 2, 'rlat': 412, 'rlon': 424, 'vertices': 4})

In [8]:
#Metadaten Klima-Modell etc.
data_pr.attrs

{'source': 'DWD-EPISODES2018',
 'institution': 'Deutscher Wetterdienst',
 'Conventions': 'CF-1.4',
 'contact': 'klima.episodes@dwd.de',
 'references': 'http://www.dwd.de/episodes',
 'institute_id': 'DWD',
 'model_id': 'DWD-EPISODES2018',
 'initialization_method': '1',
 'physics_version': '1',
 'rcm_version_id': 'v1-r1',
 'predictor_data': 'NCARv1',
 'predictor_spatial_resolution': '0.75deg',
 'predictor_typical_domain_size': '10W30E/40N60N',
 'predictor_lonrange': '10degW30degE',
 'predictor_latrange': '40degN60degN',
 'predictor_number_eofs': 'NA',
 'predictor_temporal_resolution': 'daily',
 'predictor_names': 'zg1000, zg850, zg700, ta1000, ta850, ta700, hur1000, hur850, hur700',
 'title': 'DWD-EPISODES2018 model output, forcing MPI-M-MPI-ESM-LR r2i1p1, prepared for CORDEX-ESD rcp26',
 'comment': 'CORDEX Europe ESD DWD-EPISODES2018 0.11 deg EUR-11',
 'project_id': 'CORDEX-ESD',
 'CORDEX_domain': 'EUR-11',
 'driving_experiment': 'MPI-M-MPI-ESM-LR, rcp26, r2i1p1',
 'driving_model_id': '

In [ ]:
#Useful Commands 
#ds.air.mean(dim="time").plot(x="lon")
## pull out time index 0, lat index 2, and lon index 3
# ds.air.isel(time=0, lat=2, lon=3)  #  much better than ds.air[0, 2, 3]

In [9]:
data_pr_array = data_pr['pr'] #dataarray
print(data_pr)

#variables in dataset
data_pr.data_vars
data_pr.info()
data_pr.pr[0]

# Indexierter Zugriff
#First Precipitation Data whole Germany 
# 500 Zeiteintrag (in Datum) und zehnter rat (in Grad)
# Zugriff auf Stelle in den Daten nicht auf Werte
#data_pr.pr[500].plot()




<xarray.Dataset> Size: 804MB
Dimensions:       (time: 1140, bnds: 2, rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * time          (time) datetime64[ns] 9kB 2006-01-16T12:00:00 ... 2100-12-1...
    lon           (rlat, rlon) float32 699kB ...
    lat           (rlat, rlon) float32 699kB ...
  * rlon          (rlon) float64 3kB -28.38 -28.26 -28.16 ... 17.93 18.05 18.16
  * rlat          (rlat) float64 3kB -23.38 -23.26 -23.16 ... 21.61 21.73 21.83
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds     (time, bnds) datetime64[ns] 18kB ...
    lon_bnds      (rlat, rlon, vertices) float32 3MB ...
    lat_bnds      (rlat, rlon, vertices) float32 3MB ...
    rotated_pole  int32 4B ...
    pr            (time, rlat, rlon) float32 797MB ...
Attributes: (12/38)
    source:                         DWD-EPISODES2018
    institution:                    Deutscher Wetterdienst
    Conventions:                    CF-1.4
    contact:                        klima.episodes@

<xarray.DataArray 'pr' (rlat: 412, rlon: 424)> Size: 699kB
[174688 values with dtype=float32]
Coordinates:
    time     datetime64[ns] 8B 2006-01-16T12:00:00
    lon      (rlat, rlon) float32 699kB ...
    lat      (rlat, rlon) float32 699kB ...
  * rlon     (rlon) float64 3kB -28.38 -28.26 -28.16 ... 17.93 18.05 18.16
  * rlat     (rlat) float64 3kB -23.38 -23.26 -23.16 ... 21.61 21.73 21.83
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole
    cell_methods:   time: mean

In [ ]:
# Wertebereich herausfinden

rlon = data['rlon']
rlat = data['rlat']
# Drucke die minimalen und maximalen Werte der rlon&rlat-Koordinate aus
print(f"Min rlon: {rlon.min().item()}")
print(f"Max rlon: {rlon.max().item()}")
print(f"Min rlat: {rlat.min().item()}")
print(f"Max rlat: {rlat.max().item()}")

#andere Möglichkeit für Wertebereich
data_pr.pr.coords

In [7]:
#sel select values by label 
#Slice Right Time 
data_pr_2030 = data_pr.sel(time=slice("2024-01-16","2054-01-16"))
# Slice degrees beiing out of germany
data_pr_2030_zoom = data_pr_2030.sel(rlon=slice(-9,-1), rlat=slice(-5,6))
data_pr_2030_zoom.pr.coords
data_pr_2030_zoom.pr[60].plot
#data_pr_2030_zoom.pr[5].plot()
#data_pr_2030_zoom.pr[200].plot()
data_pr_2030_zoom.pr

<xarray.DataArray 'pr' (time: 361, rlat: 100, rlon: 72)> Size: 10MB
[2599200 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 3kB 2024-01-16T12:00:00 ... 2054-01-16T12:...
    lon      (rlat, rlon) float32 29kB ...
    lat      (rlat, rlon) float32 29kB ...
  * rlon     (rlon) float64 576B -8.905 -8.795 -8.685 ... -1.315 -1.205 -1.095
  * rlat     (rlat) float64 800B -4.895 -4.785 -4.675 ... 5.775 5.885 5.995
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole
    cell_methods:   time: mean

In [ ]:
# Not working just for Test 

# Lon & Lat for Berlin 52.520008, 13.404954.
rlat_val = 300
rlon_val = 300 

# Neues Datenset für bestimmten Ort extrahieren
pr_data = data['pr'].sel(rlat=rlat_val, rlon=rlon_val, method='nearest')

# Diagramm das läuft noch nicht vor allem x-achse falscher input 
plt.figure(figsize=(0,6000))
plt.plot(data['time'], pr_data, label="Precipitation")
plt.xlabel('Time')
plt.ylabel('Precipitation (mm)')
plt.title('Precipitation over Time at rlat={}, rlon{}'.format(rlat_val,rlon_val))
plt.grid()
plt.show()

print(data)
time_data = data['time'].to_dataframe()
# von 2006 bis 2100 
print(time_data)

